In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Smart Study Assistant Project/Model for Gate questions classification/questions-data-new.csv')

In [ ]:
df.sample(5)

,topic,question
569,Operating System,A process has been allocated 3 page frames. As...
1357,Programming and Data Structure,The best data structure to check whether an ar...
1970,Digital Logic,Use Boolean laws to reduce (A+A′)(B+B′)
1394,Programming and Data Structure,"A binary search tree contains the value 1,2,3,..."
242,Computer Networks,The transport layer protocols used for real ti...


In [ ]:
df['topic'].value_counts()

,count
topic,
Computer Networks,300
Operating System,300
Mathematics,300
General Aptitude,300
Programming and Data Structure,300
Computer Organization and Architecture,300
Digital Logic,300
Theory of Computation,300


In [ ]:
df.isnull().sum()

,0
topic,0
question,0


In [ ]:
df.duplicated().sum()

90

In [ ]:
df = df.drop_duplicates()

In [ ]:
df['topic'].value_counts()

,count
topic,
Mathematics,300
Theory of Computation,296
Programming and Data Structure,293
Computer Networks,292
General Aptitude,292
Digital Logic,292
Operating System,287
Computer Organization and Architecture,258


In [ ]:
df.shape

(2310, 2)

Preprocessing:

In [ ]:
# Lowercasing
df['question'] = df['question'].str.lower()

<ipython-input-10-34e5a0f378e8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['question'] = df['question'].str.lower()


In [ ]:
# Remove punctuations
import string
def remove_punctuations(text):
  for punc in string.punctuation:
    text = text.replace(punc, '')
  return text
df['question'] = df['question'].apply(remove_punctuations)

<ipython-input-11-4a26ea8c91ad>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['question'] = df['question'].apply(remove_punctuations)


In [ ]:
df.head(5)

,topic,question
0,Computer Networks,in the following pairs of osi protocol layersu...
1,Computer Networks,an ip machine q has a path to another ip machi...
2,Computer Networks,to send same bit sequence nrz encoding require
3,Computer Networks,if there are n devices nodes in a network what...
4,Computer Networks,in networking terminology utp means


In [ ]:
# Removing stop words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
def remove_stopwords(text):
  text = [word for word in text.split() if word not in stop_words]
  return " ".join(text)
df['question'] = df['question'].apply(remove_stopwords)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df.head(5)

,topic,question
0,Computer Networks,following pairs osi protocol layersublayer fun...
1,Computer Networks,ip machine q path another ip machine h via thr...
2,Computer Networks,send bit sequence nrz encoding require
3,Computer Networks,n devices nodes network number cable links req...
4,Computer Networks,networking terminology utp means


In [ ]:
# Tokenize
import nltk
nltk.download('punkt_tab')
df['question'] = df['question'].apply(nltk.word_tokenize)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
df.sample(5)

,topic,question
228,Computer Networks,"[assuming, given, network, layer, implementati..."
1573,Computer Organization and Architecture,"[many, bits, x, fields, size, control, memory,..."
272,Computer Networks,"[consider, simple, communication, system, mult..."
2211,Theory of Computation,"[consider, dfa, ∑, b, accepting, strings, numb..."
1185,General Aptitude,"[ratio, boys, girls, class, 7, 3, among, optio..."


In [ ]:
# Lemmatization
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

def lemmatize_tokens(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]

df['question'] = df['question'].apply(lemmatize_tokens)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
df.sample(10)

,topic,question
795,Mathematics,"[proposition, ¬p→q, true, truth, value, propos..."
2316,Theory of Computation,"[let, l, w, number, substring, “, aa, ”, numbe..."
2001,Digital Logic,"[explain, role, don, ’, tcare, condition, circ..."
1933,Digital Logic,"[show, overflow, detected, binary, addition, u..."
885,Mathematics,"[fair, die, rolled, twice, find, probability, ..."
830,Mathematics,"[g, function, b, f, function, b, c, compositio..."
981,General Aptitude,"[dance, programme, scheduled, 1000, student, p..."
2042,Digital Logic,"[draw, timing, diagram, jk, flipflop, preset, ..."
2307,Theory of Computation,"[use, pumping, lemma, prove, l, aⁿbᵐcⁿ, n, ≥, ..."
233,Computer Networks,"[host, machine, us, token, bucket, algorithm, ..."


In [ ]:
# Word2Vec
from gensim.models import Word2Vec
model = Word2Vec(df['question'], vector_size=100, window=10, min_count=1)

In [21]:
# corpus build
corpus = []
for question in df['question']:
  corpus.append(question)

In [22]:
model.build_vocab(corpus)
model.train(corpus, total_examples=model.corpus_count, epochs=model.epochs)

(231169, 255510)

In [23]:
len(model.wv.index_to_key)

6286

In [29]:
# returns the average word2vec embeddings
def document_vector(doc_tokens):
    # remove out-of-vocabulary words
    words = [word for word in doc_tokens if word in model.wv.index_to_key]
    if len(words) == 0:
        return np.zeros(model.vector_size)  # fallback vector if words.size() is 0
    return np.mean(model.wv[words], axis=0)

In [33]:
from tqdm import tqdm

In [35]:
X = []
for doc in tqdm(df['question'].values):
    X.append(document_vector(doc))

100%|██████████| 2310/2310 [00:01<00:00, 2139.04it/s]


In [36]:
X = np.array(X)
X[0]

array([ 1.15132466e-01,  8.84868026e-01, -3.71105492e-01, -6.74977005e-02,
        3.51953983e-01, -1.58822513e+00, -4.12178904e-01,  2.41032267e+00,
        1.66197136e-01, -6.47583425e-01, -2.67992496e-01,  1.85188968e-02,
       -2.35867888e-01,  4.15150464e-01, -4.57992226e-01, -1.07023418e+00,
        5.51606297e-01, -1.07565641e+00,  4.52050179e-01, -2.05014372e+00,
        9.78418231e-01,  1.55204818e-01, -6.95962727e-01, -1.24595910e-01,
       -1.52621821e-01, -6.16306439e-02, -7.95515001e-01,  5.37326336e-01,
       -5.69159627e-01, -7.37400830e-01,  7.53834784e-01,  1.10294914e+00,
       -1.06303275e+00, -7.86648870e-01, -5.26814401e-01,  4.47065562e-01,
       -8.16524684e-01, -9.97634172e-01,  6.76287591e-01, -1.34244359e+00,
       -5.16539216e-02, -7.08438575e-01, -5.32241575e-02, -2.44328469e-01,
        3.36309195e-01,  2.89679497e-01, -5.21824360e-01,  9.51115191e-02,
        5.17444313e-01,  4.17762786e-01,  4.66586411e-01,  9.39820483e-02,
        9.60662007e-01, -

In [37]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

y = encoder.fit_transform(df['topic'])

In [44]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

Applying Random Forest:



In [47]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.5865800865800865

Applying Logistic Regression:

In [51]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(multi_class='multinomial')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.5432900432900433

Applying SVM:

In [50]:
from sklearn.svm import SVC
clf = SVC(kernel='linear')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.5497835497835498